# Neural Network - 2 tier approach

embeddings -> brand classification

### Imports and configuration

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU")
else:
    device = torch.device("cpu")
    print("Using CPU")

# device = torch.device("cpu")  # Debugging purposes - easier to debug with CPU

Using GPU


In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle

from sklearn.metrics import accuracy_score, precision_score, recall_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

### Load data from pkl file

In [ ]:
class SimpleDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [ ]:
folder_to_save_products_embeddings = "Tier_approach_logs_performacne"
train_embeddings = os.path.join(folder_to_save_products_embeddings, 'train_set_embeddings.pkl')
test_embeddings = os.path.join(folder_to_save_products_embeddings, 'test_set_embeddings.pkl')


In [ ]:
BATCH_SIZE = 16

# CHANGE COLUMN NAMES AS NEEDED TO LOAD DATA:

# Load embeddings and labels from the pickle file
with open(train_embeddings, 'rb') as f:
    data = pickle.load(f)
    loaded_embeddings = data['embeddings']
    loaded_master_labels = data['master_labels']
    loaded_sub_labels = data['sub_labels']
    loaded_brand_labels = data['brand_labels']

train_data = loaded_embeddings

if isinstance(train_data, torch.Tensor):
    train_data = train_data.detach().cpu().numpy()  # Convert to NumPy array

if isinstance(loaded_brand_labels, np.ndarray):
    loaded_brand_labels = torch.from_numpy(loaded_brand_labels)  # Convert to tensor

mapped_labels = False
if int(max(set(loaded_brand_labels)) + 1) != len(loaded_brand_labels.unique()):
    # Check if the mapping is off
    print(f'max label: {max(set(loaded_brand_labels))}, num of unique labels: {len(loaded_brand_labels.unique())} - remapping labels...')
    unique_labels = torch.unique(loaded_brand_labels)
    label_mapping = {old_label.item(): new_label for new_label, old_label in enumerate(unique_labels)}
    loaded_brand_labels = loaded_brand_labels.clone().apply_(lambda x: label_mapping[x])
    mapped_labels = True

train_num_brands = len(loaded_brand_labels.unique())
train_y_brand = loaded_brand_labels.cpu()

# Print to verify the loaded data
print("Embeddings shape:", loaded_embeddings.shape)  # Should show the shape of the embeddings
print("number of Unique brands:", train_num_brands)
print('max brand class: ', max(loaded_brand_labels))
print('min brand class: ', min(loaded_brand_labels))
print("train_y_brand.shape:", train_y_brand.shape)

brand_dataset_train = SimpleDataset(train_data, train_y_brand)
brand_dataloader_train = DataLoader(brand_dataset_train, batch_size=BATCH_SIZE, shuffle=True)



Embeddings shape: torch.Size([87672, 1768])
number of Unique brands: 3892
max brand class:  tensor(3891)
min brand class:  tensor(0)
train_y_brand.shape: torch.Size([87672])


C:\Users\Austin\AppData\Local\Temp\ipykernel_3812\2680378259.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels, dtype=torch.float32)


In [ ]:
# Load embeddings and labels from the pickle file

with open(test_embeddings, 'rb') as f:
    data = pickle.load(f)
    loaded_embeddings = data['embeddings']
    loaded_master_labels = data['master_labels']
    loaded_sub_labels = data['sub_labels']
    loaded_brand_labels = data['brand_labels']

test_data = loaded_embeddings

if isinstance(test_data, torch.Tensor):
    test_data = test_data.detach().cpu().numpy()  # Convert to NumPy array

if isinstance(loaded_brand_labels, np.ndarray):
    loaded_brand_labels = torch.from_numpy(loaded_brand_labels)  # Convert to tensor

if mapped_labels:
    # Apply the same label mapping to the test set if needed
    loaded_brand_labels = loaded_brand_labels.clone().apply_(lambda x: label_mapping.get(x, -1))

test_y_brand = loaded_brand_labels.cpu()

# Print to verify the loaded data
print("Embeddings shape:", loaded_embeddings.shape)  # Should show the shape of the embeddings
print("number of Unique brands:", len(loaded_brand_labels.unique()))
print('max brand: ', max(loaded_brand_labels))
print('min brand: ', min(loaded_brand_labels))
print("test_y_brand.shape:", test_y_brand.shape)


brand_dataset_test = SimpleDataset(test_data, test_y_brand)
brand_dataloader_test = DataLoader(brand_dataset_test, batch_size=BATCH_SIZE, shuffle=False)


Embeddings shape: torch.Size([21919, 1768])
number of Unique brands: 3136
max brand:  tensor(3891)
min brand:  tensor(1)
test_y_brand.shape: torch.Size([21919])


C:\Users\Austin\AppData\Local\Temp\ipykernel_3812\2680378259.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels, dtype=torch.float32)


In [ ]:
print("Train data (embeddings) shape:", train_data.shape)
print("Test data (embeddigs) shape:", test_data.shape)
print('Train label (brand) shape: ', train_y_brand.shape)
print('Test label (brand) shape: ', test_y_brand.shape)

Train data (embeddings) shape: (87672, 1768)
Test data (embeddigs) shape: (21919, 1768)
Train label (brand) shape:  torch.Size([87672])
Test label (brand) shape:  torch.Size([21919])


# Neural Network

In [ ]:
class brandClassifierNN(nn.Module):
    def __init__(self, input_size, output_size, layer1=64, layer2=32, dropout_rate=0.2, lr=0.001):
        super(brandClassifierNN, self).__init__()
        self.fc1 = nn.Linear(input_size, layer1)  # First hidden layer
        self.dropout1 = nn.Dropout(dropout_rate)   # Dropout layer after first hidden layer
        self.fc2 = nn.Linear(layer1, layer2)        # Second hidden layer
        self.dropout2 = nn.Dropout(dropout_rate)   # Dropout layer after second hidden layer
        self.fc3 = nn.Linear(layer2, output_size)   # Output layer
        self.relu = nn.ReLU()                       # Activation function
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)  # Apply dropout after the first layer
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)  # Apply dropout after the second layer
        x = self.fc3(x)
        return x

    def train_model(self, train_dataloader, val_dataloader, optimizer=None, num_epochs=100, k=10, save_directory=None, log_file=None):
        if save_directory:
            os.makedirs(save_directory, exist_ok=True)  # Create directory if it doesn't exist

        if optimizer is None:
            optimizer = self.optimizer
        self.train()  # Set the model to training mode
        for epoch in range(num_epochs):
            for features, labels in train_dataloader:
                optimizer.zero_grad()  # Clear gradients
                features = features.to(device)
                labels = labels.to(device)

                # Forward pass
                logits = self(features)
                labels = labels.long()
                loss = self.criterion(logits, labels)  # Reshape if needed

                # Backward pass
                loss.backward()
                optimizer.step()  # Update weights

            if epoch % k == 0 or epoch == num_epochs - 1:
                # Evaluate on validation set
                _, val_loss, val_accuracy, strict_val_accuracy = self.evaluate(val_dataloader)
                self.train()

                log_message = (f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, "
                           f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, "
                           f"99% Accuracy: {strict_val_accuracy:.4f}")
                print(log_message)

                # If a log file is specified, append to it
                if save_directory:
                    if log_file:
                        log_path = os.path.join(save_directory, log_file)

                        if log_path.endswith(".txt"):  # Ensure it's a text file
                            with open(log_path, "a") as f:  # Open in append mode
                                f.write(log_message + "\n")
                        else:
                            print("Warning: Provided file is not a .txt file, skipping logging.")
                    file_name = os.path.join(save_directory, f"model.pth")
                    # Save the model
                    torch.save(self, file_name)

        return val_loss

    def evaluate(self, dataloader, device=device):
        self.eval()  # Set the model to evaluation mode
        total_loss = 0
        correct_predictions = 0
        strict_correct_predictions = 0
        total_samples = 0
        all_logits = []

        with torch.no_grad():  # No need to track gradients
            for features, labels in dataloader:
                features = features.to(device)
                labels = labels.to(device)

                # Forward pass
                logits = self(features)  # Get the raw logits
                all_logits.append(logits.cpu())
                labels = labels.long()

                # Compute loss
                loss = self.criterion(logits, labels)
                total_loss += loss.item()

                # Calculate accuracy
                probabilities = torch.softmax(logits, dim=1)
                predictions = torch.argmax(probabilities, axis=1)
                correct_predictions += (predictions == labels).sum().item()

                strict_predictions = (probabilities.max(dim=1).values >= 0.99) & (predictions == labels)
                strict_correct_predictions += strict_predictions.sum().item()

                total_samples += labels.size(0)

        all_logits = torch.cat(all_logits, dim=0)
        avg_loss = total_loss / len(dataloader)
        accuracy = correct_predictions / total_samples
        strict_accuracy = strict_correct_predictions / total_samples
        return all_logits, avg_loss, accuracy, strict_accuracy

### Hyperparameter Tuning

In [ ]:
def random_search(train_loader, val_loader, n_trials=10, epochs=10, k=5):
    best_model = None
    best_loss = float('inf')
    best_params = {}

    for i in range(n_trials):

        # CHANGE SEARCH PARAMETERS AS NEEDED:

        layer1 = np.random.randint(64, 1024)
        layer2 = np.random.randint(32, 1024)
        dropout_rate = np.random.uniform(0.05, 0.4)
        learning_rate = 10**np.random.uniform(-6, -2)

        # layer1 = np.random.randint(32, 128)  # Number of neurons in first layer
        # layer2 = np.random.randint(16, 64)   # Number of neurons in second layer
        # dropout_rate = np.random.uniform(0.1, 0.5)  # Dropout rate
        # learning_rate = 10**np.random.uniform(-5, 0)  # Learning rate

        print(f'--- Params (iter {i+1}) ---')
        print(f'layer1: {layer1}, layer2: {layer2}, dropout: {dropout_rate}, learning_rate: {learning_rate}')

        # Initialize model, loss function, and optimizer
        embedding_size = train_data.shape[1]
        model = brandClassifierNN(embedding_size, train_num_brands, layer1=layer1, layer2=layer2, dropout_rate=dropout_rate)
        model.to(device)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        # Train model
        val_loss = model.train_model(train_dataloader=train_loader, val_dataloader=val_loader, optimizer=optimizer, num_epochs=epochs, k=k)

        # Update best model if current one is better
        if val_loss < best_loss:
            best_loss = val_loss
            best_model = model
            best_params = {'layer1': layer1, 'layer2': layer2, 'dropout_rate': dropout_rate, 'learning_rate': learning_rate}

    return best_model, best_params, best_loss


### Neural Network Training

In [ ]:
# Hyperparam tuning
best_model, best_params, best_loss = random_search(brand_dataloader_train, brand_dataloader_test, n_trials=6, epochs=5, k=1)

print(f'Best Loss: {best_loss}')
print(f'Best Hyperparameters: {best_params}')


# Initialize model
logfilepath = "logs.txt"
current_time = datetime.now().strftime("%Y-%m-%d_%I-%M_%p")
train_log_title = 'Filename' #TODO: INSERT FILE NAME HERE
title = train_log_title + "_" if train_log_title else ''
save_directory = f"{title}train_logs_{current_time}"

embedding_size = train_data.shape[1]

# Can also manually set number of nodes and parameters. For example:
# model_brand = brandClassifierNN(embedding_size, train_num_brands, layer1=249, layer2=433, dropout_rate=0.12155841326717244, lr=0.00008654271868553579)

model_brand = brandClassifierNN(
    embedding_size,
    train_num_brands,
    layer1=best_params['layer1'],
    layer2=best_params['layer2'],
    dropout_rate=best_params['dropout_rate'],
    lr=best_params['learning_rate']
    )

model_brand.to(device)
num_weights = sum(p.numel() for p in model_brand.parameters() if p.requires_grad)
print(f"Number of learnable weights: {num_weights}")


# Train the model

# For enabling training logs:
model_brand.train_model(brand_dataloader_train, brand_dataloader_test, num_epochs=50, k=1, save_directory=save_directory, log_file=logfilepath)

# For disabling traning logs:
# model_brand.train_model(brand_dataloader_train, brand_dataloader_test, num_epochs=50, k=1)

logits_brand, loss_brand, accuracy_brand, strict_accuracy_brand = model_brand.evaluate(brand_dataloader_test)
logits_brand = logits_brand.cpu().numpy()


--- Params (iter 0) ---
layer1: 823, layer2: 150, dropout: 0.2384689883394433, learning_rate: 0.002335975404451696
Epoch [1/1], Loss: 5.4217, Val Loss: 5.5983, Val Accuracy: 0.0939, 99% Accuracy: 0.0121
Best Loss: 5.598326482215937
Best Hyperparameters: {'layer1': 823, 'layer2': 150, 'dropout_rate': 0.2384689883394433, 'learning_rate': 0.002335975404451696}
Number of learnable weights: 2167179
Epoch [1/5], Loss: 6.4838, Val Loss: 5.8245, Val Accuracy: 0.1002, 99% Accuracy: 0.0183
Epoch [2/5], Loss: 6.7148, Val Loss: 5.6035, Val Accuracy: 0.1011, 99% Accuracy: 0.0326
Epoch [3/5], Loss: 5.8328, Val Loss: 5.6515, Val Accuracy: 0.0910, 99% Accuracy: 0.0317
Epoch [4/5], Loss: 5.0018, Val Loss: 5.6073, Val Accuracy: 0.0911, 99% Accuracy: 0.0334
Epoch [5/5], Loss: 5.7651, Val Loss: 5.5462, Val Accuracy: 0.1022, 99% Accuracy: 0.0411


### Neural Network Evaluation

In [ ]:
def get_threshold_metrics(logits, y_true, threshold=None):
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()
    y_pred = np.argmax(logits, axis=1)

    unique_images_test = len(y_true)  # Unique images in the test set
    unique_brands_test = len(np.unique(y_true))  # Unique brands in the test set

    if threshold:
        predicted_probabilities = probs[np.arange(len(probs)), y_pred]
        y_pred_threshold = np.where(predicted_probabilities >= threshold, y_pred, -1)

        accuracy = accuracy_score(y_true, y_pred_threshold)
        precision = precision_score(y_true, y_pred_threshold, average='macro', zero_division=0)
        recall = recall_score(y_true, y_pred_threshold, average='macro', zero_division=0)
        f1 = (2 * precision * recall) / (precision + recall) if (precision+recall>0) else 0.0

        valid_indices = predicted_probabilities >= threshold  # Get indices where predictions meet the threshold
        y_pred_adj =  y_pred_threshold[valid_indices]
        adj_accuracy = accuracy_score(y_true[valid_indices], y_pred_adj) if np.any(valid_indices) else 0.0
        unique_images_pred = len(y_pred_adj)
        unique_brands_pred = len(np.unique(y_pred_adj))

    else:
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, average='macro')
        recall = recall_score(y_true, y_pred, average='macro')
        f1 = (2 * precision * recall) / (precision + recall)
        adj_accuracy = accuracy
        unique_images_pred = len(y_pred)
        unique_brands_pred = len(np.unique(y_pred))

    coverage_images = unique_images_pred / unique_images_test if unique_images_test > 0 else 0.0
    coverage_brands = unique_brands_pred / unique_brands_test if unique_brands_test > 0 else 0.0

    return accuracy, precision, recall, f1, adj_accuracy, coverage_images, coverage_brands


def evaluate_and_save_model_stats(logits, y_true, metrics_title, file_path=None, print_metrics=False):
    # Dictionary to store metrics for each threshold
    metrics_dict = {
        "Threshold": [],
        "Accuracy": [],
        "Precision": [],
        "Recall": [],
        "F1 Score": [],
        "Adjusted Accuracy": [],
        "Image Coverage": [],
        "Brand Coverage": []
    }

    # Evaluate metrics at default, 99%, and 95% thresholds
    for threshold, label in [(None, "Default"), (0.95, "95%"), (0.96, "96%"), (0.97, "97%"), (0.98, "98%"), (0.99, "99%")]:
        accuracy, precision, recall, f1, adj_accuracy, cov_img, cov_brand = get_threshold_metrics(logits, y_true, threshold)

        accuracy = round(accuracy, 3)
        precision = round(precision, 3)
        recall = round(recall, 3)
        f1 = round(f1, 3)
        adj_accuracy = round(adj_accuracy, 3)
        cov_img = round(cov_img, 3)
        cov_brand = round(cov_brand, 3)

        if print_metrics:
            # Print metrics to console
            print(f'\n--- {metrics_title} {label} Metrics ---')
            print(f'Accuracy: {accuracy}')
            print(f'Precision: {precision}')
            print(f'Recall: {recall}')
            print(f'F1 Score: {f1}')
            print(f'Adjusted Accuracy: {adj_accuracy}')
            print(f'Image Coverage: {cov_img}')
            print(f'Brand Coverage: {cov_brand}')

            # Store metrics in dictionary
            metrics_dict["Threshold"].append(label)
            metrics_dict["Accuracy"].append(accuracy)
            metrics_dict["Precision"].append(precision)
            metrics_dict["Recall"].append(recall)
            metrics_dict["F1 Score"].append(f1)
            metrics_dict["Adjusted Accuracy"].append(adj_accuracy)
            metrics_dict["Image Coverage"].append(cov_img)
            metrics_dict["Brand Coverage"].append(cov_brand)

    if file_path:
        metrics_df = pd.DataFrame(metrics_dict)

        definitions = {
        "Metric": [
            "Softmax Threshold",
            "Accuracy",
            "Adjusted Accuracy",
            "Coverage_images",
            "Coverage_brands"
        ],
        "Definition": [
            "No threshold: all images are used. 99% threshold: predictions meeting at least 99% softmax probability.",
            "Number of predictions meeting the threshold and are correct / test set size.",
            "Number of predictions meeting the threshold and are correct / Number of predictions meeting the threshold.",
            "Unique images meeting the threshold / unique images in test set.",
            "Unique brands meeting the threshold / unique brands in test set."
        ],
        "Example": [
            "100 images with 85 unique brands. Model classified 90 images correctly with no threshold. 80 images (55 unique brands) meet the 99% threshold, out of these 80 images 75 were classified correctly.",
            "No threshold accuracy: 0.90, 99% threshold accuracy: 0.75",
            "99% adjusted accuracy: 75/80 = 0.93",
            "99% Coverage_Images: 0.80",
            "99% Coverage_brands: 55/85 = 0.647"
        ]
        }
        definitions_df = pd.DataFrame(definitions)


        with pd.ExcelWriter(file_path) as writer:
            metrics_df.to_excel(writer, sheet_name="Metrics", index=False)
            definitions_df.to_excel(writer, sheet_name="Definitions", index=False)

        print(f"Metrics and definitions saved to {file_path}")

file_name = f"{train_log_title}_brand"
file_name = file_name + '_' if file_name else ''
file_path = f'{save_directory}\\{file_name}metrics_output.xlsx'
evaluate_and_save_model_stats(logits_brand, test_y_brand, metrics_title=file_name, file_path=file_path, print_metrics=True)


c:\Users\Austin\anaconda3\envs\condaenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



--- TestTest_brand_ Default Metrics ---
Accuracy: 0.102
Precision: 0.011
Recall: 0.012
F1 Score: 0.012
Adjusted Accuracy: 0.102
Image Coverage: 1.0
Brand Coverage: 0.03

--- TestTest_brand_ 95% Metrics ---
Accuracy: 0.045
Precision: 0.009
Recall: 0.006
F1 Score: 0.007
Adjusted Accuracy: 0.986
Image Coverage: 0.045
Brand Coverage: 0.01

--- TestTest_brand_ 96% Metrics ---
Accuracy: 0.044
Precision: 0.009
Recall: 0.005
F1 Score: 0.007
Adjusted Accuracy: 0.987
Image Coverage: 0.044
Brand Coverage: 0.01

--- TestTest_brand_ 97% Metrics ---
Accuracy: 0.043
Precision: 0.009
Recall: 0.005
F1 Score: 0.007
Adjusted Accuracy: 0.986
Image Coverage: 0.044
Brand Coverage: 0.009

--- TestTest_brand_ 98% Metrics ---
Accuracy: 0.042
Precision: 0.009
Recall: 0.005
F1 Score: 0.007
Adjusted Accuracy: 0.986
Image Coverage: 0.043
Brand Coverage: 0.009

--- TestTest_brand_ 99% Metrics ---
Accuracy: 0.041
Precision: 0.009
Recall: 0.005
F1 Score: 0.006
Adjusted Accuracy: 0.987
Image Coverage: 0.042
Brand Cov